In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q --use-deprecated=legacy-resolver tflite-model-maker
!pip install -q pycocotools

     |████████████████████████████████| 591 kB 8.7 MB/s 
     |████████████████████████████████| 1.1 MB 50.8 MB/s 
     |████████████████████████████████| 103 kB 52.5 MB/s 
     |████████████████████████████████| 120 kB 54.5 MB/s 
     |████████████████████████████████| 636 kB 54.7 MB/s 
     |████████████████████████████████| 840 kB 40.2 MB/s 
     |████████████████████████████████| 64 kB 3.6 MB/s 
     |████████████████████████████████| 6.3 MB 38.2 MB/s 
     |████████████████████████████████| 1.2 MB 39.3 MB/s 
     |████████████████████████████████| 211 kB 56.2 MB/s 
     |████████████████████████████████| 87 kB 7.0 MB/s 
     |████████████████████████████████| 1.1 MB 41.5 MB/s 
     |████████████████████████████████| 99 kB 8.2 MB/s 
     |████████████████████████████████| 352 kB 55.0 MB/s 
     |████████████████████████████████| 37.1 MB 1.2 MB/s 
     |████████████████████████████████| 200 kB 53.1 MB/s 


In [3]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [4]:
!rm -rf '/content/data'

In [5]:
# !mkdir /content/data/

In [6]:
# !curl -L "https://app.roboflow.com/ds/eWKgcOFrno?key=kHodCIYX8C" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [7]:
cp '/content/drive/MyDrive/1/pie/Annotations/bigimages.v35-og2_aug2.voc.zip' -d '/content/bigimages.v35-og2_aug2.voc.zip'

In [8]:
!unzip '/content/bigimages.v35-og2_aug2.voc.zip' -d '/content/data/'

Streaming output truncated to the last 5000 lines.
 extracting: /content/data/train/two_g_64_jpg.rf.e81812ebe5e3cc63e7c5332b04e706f4.jpg  
 extracting: /content/data/train/two_g_64_jpg.rf.e81812ebe5e3cc63e7c5332b04e706f4.xml  
 extracting: /content/data/train/two_g_65_jpg.rf.1189f2bb0d211f6feea0619a07ff34db.jpg  
 extracting: /content/data/train/two_g_65_jpg.rf.1189f2bb0d211f6feea0619a07ff34db.xml  
 extracting: /content/data/train/two_g_65_jpg.rf.42b2052fed4b8898c7d385c280aab5a6.jpg  
 extracting: /content/data/train/two_g_65_jpg.rf.42b2052fed4b8898c7d385c280aab5a6.xml  
 extracting: /content/data/train/two_g_65_jpg.rf.a87b0b8f96d5979b7ea5adb6ea6cad97.jpg  
 extracting: /content/data/train/two_g_65_jpg.rf.a87b0b8f96d5979b7ea5adb6ea6cad97.xml  
 extracting: /content/data/train/two_g_66_jpg.rf.95370b16bbe04fcb57aa264915b04d01.jpg  
 extracting: /content/data/train/two_g_66_jpg.rf.95370b16bbe04fcb57aa264915b04d01.xml  
 extracting: /content/data/train/two_g_66_jpg.rf.a33ae205a5c26f4313a5

In [9]:
# !cp -r '/content/train' -d '/content/data/'

In [10]:
spec = model_spec.get('efficientdet_lite1')

In [11]:
spec.config.tflite_max_detections=4
# spec.config.learning_rate=1e-3

In [12]:
labels  = {}
a_file = open("/content/drive/MyDrive/1/pie/Annotations/labels.txt")

for line_number, line in enumerate(a_file.readlines()):
    labels[line_number+1] = line.rstrip("\n")

print(labels)

{1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: 'A', 11: 'B', 12: 'Bullseye', 13: 'C', 14: 'D', 15: 'Down', 16: 'E', 17: 'F', 18: 'G', 19: 'H', 20: 'Left', 21: 'Right', 22: 'S', 23: 'T', 24: 'U', 25: 'Up', 26: 'V', 27: 'W', 28: 'X', 29: 'Y', 30: 'Yellow_Circle', 31: 'Z'}


In [13]:
labels

{1: '1',
 2: '2',
 3: '3',
 4: '4',
 5: '5',
 6: '6',
 7: '7',
 8: '8',
 9: '9',
 10: 'A',
 11: 'B',
 12: 'Bullseye',
 13: 'C',
 14: 'D',
 15: 'Down',
 16: 'E',
 17: 'F',
 18: 'G',
 19: 'H',
 20: 'Left',
 21: 'Right',
 22: 'S',
 23: 'T',
 24: 'U',
 25: 'Up',
 26: 'V',
 27: 'W',
 28: 'X',
 29: 'Y',
 30: 'Yellow_Circle',
 31: 'Z'}

In [14]:
import os

os.rename('/content/data/train', '/content/data/images')

In [15]:
import os, random, shutil

os.mkdir('/content/data/train')
os.mkdir('/content/data/valid')

image_paths = os.listdir('/content/data/images')
random.shuffle(image_paths)

for i, image_path in enumerate(image_paths):
  if i < int(len(image_paths) * 0.8):
    shutil.copy(f'/content/data/images/{image_path[:-3]}'+'jpg', '/content/data/train')
    shutil.copy(f'/content/data/images/{image_path[:-3]}'+'xml', '/content/data/train')
  else:
    shutil.copy(f'/content/data/images/{image_path[:-3]}'+'jpg', '/content/data/valid')
    shutil.copy(f'/content/data/images/{image_path[:-3]}'+'xml', '/content/data/valid')

In [16]:
train_data = object_detector.DataLoader.from_pascal_voc(images_dir='/content/data/train', annotations_dir='/content/data/train', label_map=labels)

In [17]:
val_data = object_detector.DataLoader.from_pascal_voc(images_dir='/content/data/valid', annotations_dir='/content/data/valid', label_map=labels)

In [18]:
train_data

In [19]:
model = object_detector.create(train_data, model_spec=spec, epochs=120, batch_size=8, train_whole_model=True, validation_data=val_data)

Epoch 1/120
1103/1103 [==============================] - 466s 375ms/step - det_loss: 1.1065 - cls_loss: 0.8569 - box_loss: 0.0050 - reg_l2_loss: 0.0768 - loss: 1.1833 - learning_rate: 0.0090 - gradient_norm: 3.3951 - val_det_loss: 0.7221 - val_cls_loss: 0.5067 - val_box_loss: 0.0043 - val_reg_l2_loss: 0.0774 - val_loss: 0.7995
Epoch 2/120
1103/1103 [==============================] - 410s 372ms/step - det_loss: 0.6313 - cls_loss: 0.4657 - box_loss: 0.0033 - reg_l2_loss: 0.0779 - loss: 0.7092 - learning_rate: 0.0100 - gradient_norm: 2.8196 - val_det_loss: 0.5495 - val_cls_loss: 0.3991 - val_box_loss: 0.0030 - val_reg_l2_loss: 0.0784 - val_loss: 0.6279
Epoch 3/120
1103/1103 [==============================] - 409s 371ms/step - det_loss: 0.5121 - cls_loss: 0.3599 - box_loss: 0.0030 - reg_l2_loss: 0.0788 - loss: 0.5909 - learning_rate: 0.0100 - gradient_norm: 2.5862 - val_det_loss: 0.5255 - val_cls_loss: 0.3770 - val_box_loss: 0.0030 - val_reg_l2_loss: 0.0791 - val_loss: 0.6046
Epoch 4/120
1

In [20]:
model.evaluate(val_data)

52/52 [==============================] - 177s 2s/step



{'AP': 0.77704847,
 'AP50': 0.99525565,
 'AP75': 0.9346261,
 'AP_/1': 0.70593584,
 'AP_/2': 0.73135877,
 'AP_/3': 0.77201885,
 'AP_/4': 0.7386664,
 'AP_/5': 0.7297368,
 'AP_/6': 0.79764634,
 'AP_/7': 0.7526671,
 'AP_/8': 0.79105705,
 'AP_/9': 0.76933205,
 'AP_/A': 0.7576367,
 'AP_/B': 0.791033,
 'AP_/Bullseye': 0.8370025,
 'AP_/C': 0.82508326,
 'AP_/D': 0.72215194,
 'AP_/Down': 0.7909652,
 'AP_/E': 0.85588676,
 'AP_/F': 0.7701648,
 'AP_/G': 0.7580739,
 'AP_/H': 0.74235934,
 'AP_/Left': 0.8368152,
 'AP_/Right': 0.8539453,
 'AP_/S': 0.7534126,
 'AP_/T': 0.6976723,
 'AP_/U': 0.7972162,
 'AP_/Up': 0.7695652,
 'AP_/V': 0.7786339,
 'AP_/W': 0.8614844,
 'AP_/X': 0.73494256,
 'AP_/Y': 0.7814195,
 'AP_/Yellow_Circle': 0.86078274,
 'AP_/Z': 0.72383523,
 'APl': 0.9042508,
 'APm': 0.86196387,
 'APs': 0.7377615,
 'ARl': 0.92572916,
 'ARm': 0.9037751,
 'ARmax1': 0.7998426,
 'ARmax10': 0.8352078,
 'ARmax100': 0.83554447,
 'ARs': 0.8068641}

In [21]:
# from tflite_model_maker.config import QuantizationConfig

In [22]:
# config = QuantizationConfig.for_dynamic()

In [23]:
model.export(export_dir='/content/EffDetLite1_og2_aug2')

In [24]:
# model.export(export_dir='.', tflite_filename='EFfDetLite0_dynamic.tflite', quantization_config=config)

In [25]:
# cp '/content/EFfDetLite0_uint8.tflite' -d '/content/drive/MyDrive/EffDetLite0_uint8.tflite'

In [26]:
model.export(export_dir='/content/EffDetLite1_og2_aug2', export_format=[ExportFormat.SAVED_MODEL, ExportFormat.LABEL])

In [27]:
# cp '/content/EffDetLite0_uint8/model.tflite' -d '/content/drive/MyDrive/1/pie/lite0_noquant.tflite'

In [28]:
import shutil

shutil.make_archive('/content/drive/MyDrive/1/pie/Models/EffDetLite1_og2_aug2/EffDetLite1_og2_aug2', 'zip', '/content/EffDetLite1_og2_aug2/')

'/content/drive/MyDrive/1/pie/Models/EffDetLite1_og2_aug2/EffDetLite1_og2_aug2.zip'

In [29]:
# !kill -9 -1

In [30]:
# cp '/content/EffDetLite3/model.tflite' -d '/content/drive/MyDrive/1/pie/EffDetLite3.tflite'

In [31]:
# !pip install tensorflow==2.5.0

In [32]:
# model.evaluate_tflite('/content/EffDetLite0/model.tflite', val_data)

In [33]:
# TPU Edge

!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  49745      0 --:--:-- --:--:-- --:--:-- 49745
OK


In [34]:
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list



deb https://packages.cloud.google.com/apt coral-edgetpu-stable main


In [35]:
!sudo apt-get update



Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [66.6 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 https://packages.cloud.google.com/apt coral-edgetpu-stable InRelease [6,722 B]
Get:13 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:14 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get

In [36]:
!sudo apt-get install edgetpu-compiler

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  edgetpu-compiler
0 upgraded, 1 newly installed, 0 to remove and 57 not upgraded.
Need to get 7,913 kB of archives.
After this operation, 31.2 MB of additional disk space will be used.
Get:1 https://packages.cloud.google.com/apt coral-edgetpu-stable/main amd64 edgetpu-compiler amd64 16.0 [7,913 kB]
Fetched 7,913 kB in 1s (11.1 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back 

In [37]:
cp '/content/EffDetLite1_og2_aug2/model.tflite' -d '/content/model.tflite'

In [38]:
!edgetpu_compiler /content/model.tflite --num_segments=1


Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 3938 ms.

Input model: /content/model.tflite
Input size: 5.69MiB
Output model: model_edgetpu.tflite
Output size: 7.39MiB
On-chip memory used for caching model parameters: 6.05MiB
On-chip memory remaining for caching model parameters: 1.46MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 322
Operation log: model_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 315
Number of operations that will run on CPU: 7
See the operation log file for individual operation details.
Compilation child pro

In [39]:
!cp '/content/model_edgetpu.log' -d '/content/drive/MyDrive/1/pie/Models/EffDetLite1_og2_aug2/model_edgetpu.log'

In [40]:
!cp '/content/model_edgetpu.tflite' -d '/content/drive/MyDrive/1/pie/Models/EffDetLite1_og2_aug2/model_edgetpu.tflite'